In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import urllib
import zipfile
import seaborn as sns

import torch
dtype = torch.float
device = torch.device("cuda:0") 
from torch.distributions import normal
import matplotlib.pyplot as plt, numpy as np, pickle
from scipy.stats import norm
from tifffile import imread
import sys
sys.path.append('/home/ashesh.ashesh/code/PPN2V')
from src.ppn2v.pn2v import *
import src.ppn2v.pn2v.gaussianMixtureNoiseModel
import src.ppn2v.pn2v.histNoiseModel
import src.ppn2v.pn2v.prediction
from src.ppn2v.pn2v.utils import *
from src.scripts.train_n2v import get_bestmodelname, load_data
from disentangle.core.psnr import RangeInvariantPsnr



In [ ]:
ls /home/ashesh.ashesh/training/N2V/2307/17/

In [ ]:
!ls /home/ashesh.ashesh/training/N2V/2307/20/

In [ ]:

modelpath = '/home/ashesh.ashesh/training/N2V/2307/20/best_-20190520_tl_25um_50msec_05pc_488_130EM_Conv-n2v.net'

# data_dir = '/group/jug/ashesh/data/BSD68_reproducibility_data/test/'
data_dir = '/home/ashesh.ashesh/code/DivNoising/examples/Convallaria/data/Convallaria_diaphragm/'
data_dir = data_dir[:-1] if '/' == data_dir[-1] else data_dir

# low_snr_filename =  'bsd68_gaussian25.npy'
# high_snr_filename = 'bsd68_groundtruth.npy'
low_snr_filename = '20190520_tl_25um_50msec_05pc_488_130EM_Conv.tif'
high_snr_filename = '20190726_tl_50um_500msec_wf_130EM_FD.tif'
low_snr_data = load_data(os.path.join(data_dir, low_snr_filename))
high_snr_data = load_data(os.path.join(data_dir, high_snr_filename))

In [ ]:
import numpy as np


In [ ]:
net = torch.load(modelpath)

In [ ]:
from tqdm import tqdm 
results=[]
meanRes=[]
resultImgs=[]
inputImgs=[]
dataTest = low_snr_data

for index in tqdm(range(dataTest.shape[0])):

    im=dataTest[index]
    # We are using tiling to fit the image into memory
    # If you get an error try a smaller patch size (ps)
    means = src.ppn2v.pn2v.prediction.tiledPredict(im, net, ps=256, overlap=48,
                                            device=device, noiseModel=None)
    resultImgs.append(means)
    inputImgs.append(im)
    

In [ ]:
_,ax = plt.subplots(figsize=(18,6),ncols=3)
img_idx = np.random.randint(0,high=len(inputImgs))
hs = 200
ws = 200
hN = 200
wN = 200
ax[0].imshow(inputImgs[img_idx][hs:hs+hN, ws:ws+wN])
ax[1].imshow(resultImgs[img_idx][hs:hs+hN, ws:ws+wN])
ax[2].imshow(high_snr_data[img_idx][hs:hs+hN, ws:ws+wN])

In [ ]:
from skimage.io import imread, imsave

pred = np.array(resultImgs)
fpath = os.path.join('/home/ashesh.ashesh/code/DivNoising/examples/Convallaria/pseudo_gt/',low_snr_filename) 
imsave(fpath, pred,plugin='tifffile')
print(fpath)

In [ ]:
_ = plt.hist(inputImgs[img_idx].reshape(-1,),bins=50, color='red')
_ = plt.hist(resultImgs[img_idx].reshape(-1,),bins=50, color='blue')